### Import Relevant Libraries

In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import emoji
import re
import matplotlib.pyplot as plt
import os

In [2]:
# Consumer Keys
api_key = "X7zlBLEzaxfhuMOWaYNeH35nj"
api_key_secret = "s8pIdh416pp1hfMrUMfHzzDnZvYLeupR4PDvDfwWRh0Bfx2nL9"

# Authentication Tokens
access_token = "1543790938195369985-jABnbwkejQBsYIgisCSv5uIJsdMj8V"
access_token_secret = "uZVuVpqwG1HySwUXIcs1qkSazVW8WPRNj1HFWsmK1NgQh"

### Create the  Authentication Object

In [3]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

# Create the API object
api = tweepy.API(auth, wait_on_rate_limit=True)

### Extract tweets from Twitter

In [4]:
try:
    tweets = api.search_tweets(q="jakarta")
except Exception as e:
    print(e)

403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve


**Note:**<br>
Sel di atas berjalan apabila peserta memiliki akses **Elevated**. Namun, peserta menggunakan akses **Essential** untuk melakukan ekstraksi data dari Twitter API v2 sehingga sel di atas menghasilkan error.<br><br>
Akan tetapi, hal tersebut dapat diatasi dengan menggunakan **Postman API** dan **Pentaho Integration Tools** untuk ekspor data Twitter API dalam bentuk csv, yang mana dapat dilihat pada sel berikut.

### Import Data

Tanggal pengerjaan: **7 November 2022** <br>

Data yang digunakan merupakan hasil ekstraksi data dari Twitter API mulai dari **7 hari sebelum tanggal pengerjaan** hingga **tanggal pengerjaan**. <br>

* Keterbatasan data tersebut terjadi karena pengguna memperoleh akses **"Essential"** daripada **"Elevated"** dan **"Academic Research"**

In [5]:
# Isi dari folder Data/Twitter

src = './Data/Twitter'
os.listdir(src)

['goto_count_202211091623.csv',
 'shopee_count_202211091616.csv',
 'tweets_twitter_202211071059.csv',
 'twitter.txt']

In [6]:
# Full path dari data teks twitter dalam ekstensi *csv
src_csv = os.path.join(src, 'tweets_twitter_202211071059.csv')

data = pd.read_csv(src_csv)
data.head()

,text
0,PHK masal | Serba-serbi MMC\nPerusahaan besar ...
1,"@technoblase Gatau deh, Kaz. Semenjak berita y..."
2,@sicepat_ekspres kurir nya semua di PHK kah? M...
3,Biang Kerok Shopee Tutup &amp; PHK di Mana-Man...
4,RT @amorteniava: @worksfess Lah itu shopee bar...


In [7]:
# Masukkan kalimat-kalimat pada csv tersebut ke dalam list
sentences = []
for text in data['text']:
    sentences.append(text)
    
for idx, sentence in enumerate(sentences):
    print(f"{idx+1}) {sentence}\n")

1) PHK masal | Serba-serbi MMC
Perusahaan besar yang baru saja melakukan PHK massal adalah Shopee, Tokocrypto, dan Indosat. Selain itu, ada perusahaan otomotif PT Nozomi yang berujung pada demonstrasi para pegawainya. 
https://t.co/MP48oV7Bmw
#Islamkaffah

2) @technoblase Gatau deh, Kaz. Semenjak berita yg shopee phk banyak pekerjanya jd makin kesini makin kesana

3) @sicepat_ekspres kurir nya semua di PHK kah? Mau request pickup, kurir yg biasa pickup pd bilang mereka semua di PHK, ini gimana ga? Banyak yg blm dipickup, sayang nya di shopee ga bisa ganti expedisi

4) Biang Kerok Shopee Tutup &amp; PHK di Mana-Mana Ternyata di India https://t.co/0A4YodNNcz

5) RT @amorteniava: @worksfess Lah itu shopee baru PHK karyawan nya , banyak pabrik yang PHK karyawan nya gak di itung ? Selama masa pandemi b…

6) RT @wony4everyoung: @worksfess Shopee bakar uangnya daripada ngundang artis Kpop, mending banyakin voucher dan gratis ongkir kayak dulu. Ta…

7) @worksfess Lah itu shopee baru PHK karyaw

### Data Cleaning

In [8]:
def clean_text(text):
    # Menghilangkan @account
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    
    # Menghilangkan tag #
    text = re.sub('#\s?\S+', '', text)
    
    # Menghilang RT
    text = re.sub('RT[\s]+:?', '', text)
    
    # Menghilangkan hyperlink
    text = re.sub(r"https?:\/\/\S+", "", text)
    
    # Menghilangkan new enter
    text = text.replace("\n", " ")
    
    # Menghilangkan kata lainnya
    text = text.replace("&amp", "")
    
    return text.strip()

In [9]:
clean_data = data.copy()
clean_data['text'] = clean_data['text'].apply(clean_text)
clean_data.head()

,text
0,PHK masal | Serba-serbi MMC Perusahaan besar y...
1,"Gatau deh, Kaz. Semenjak berita yg shopee phk ..."
2,kurir nya semua di PHK kah? Mau request pickup...
3,Biang Kerok Shopee Tutup ; PHK di Mana-Mana Te...
4,"Lah itu shopee baru PHK karyawan nya , banyak ..."


In [10]:
# Masukkan kalimat-kalimat pada csv tersebut ke dalam list
sentences = []
for text in clean_data['text']:
    sentences.append(text)
    
for idx, sentence in enumerate(sentences):
    print(f"{idx+1}) {sentence}\n")

1) PHK masal | Serba-serbi MMC Perusahaan besar yang baru saja melakukan PHK massal adalah Shopee, Tokocrypto, dan Indosat. Selain itu, ada perusahaan otomotif PT Nozomi yang berujung pada demonstrasi para pegawainya.

2) Gatau deh, Kaz. Semenjak berita yg shopee phk banyak pekerjanya jd makin kesini makin kesana

3) kurir nya semua di PHK kah? Mau request pickup, kurir yg biasa pickup pd bilang mereka semua di PHK, ini gimana ga? Banyak yg blm dipickup, sayang nya di shopee ga bisa ganti expedisi

4) Biang Kerok Shopee Tutup ; PHK di Mana-Mana Ternyata di India

5) Lah itu shopee baru PHK karyawan nya , banyak pabrik yang PHK karyawan nya gak di itung ? Selama masa pandemi b…

6) Shopee bakar uangnya daripada ngundang artis Kpop, mending banyakin voucher dan gratis ongkir kayak dulu. Ta…

7) Lah itu shopee baru PHK karyawan nya , banyak pabrik yang PHK karyawan nya gak di itung ? Selama masa pandemi banyak orang gak kerja kena PHK massal . Lu gak tau banyak Perusahaan Gulung Tikar . L

In [11]:
# Label yang diberikan oleh peserta (diasumsikan label yang sebenarnya)

true_labels = ["Netral", 
               "Negatif", 
               "Negatif", 
               "Negatif", 
               "Negatif", 
               "Negatif", 
               "Negatif", 
               "Negatif", 
               "Netral", 
               "Negatif", 
               "Negatif", 
               "Netral"]

In [12]:
def translate_en_AND_get_polarity(text):
    # Demojize the emoji
    converted_text = emoji.demojize(text)
    
    # Translate to english
    analysis = TextBlob(converted_text)
    analysis = analysis.translate(from_lang="id", to='en')
    return analysis.sentiment.polarity

clean_data['Polarity'] = clean_data['text'].apply(translate_en_AND_get_polarity)
clean_data

,text,Polarity
0,PHK masal | Serba-serbi MMC Perusahaan besar y...,0.000000
1,"Gatau deh, Kaz. Semenjak berita yg shopee phk ...",0.500000
2,kurir nya semua di PHK kah? Mau request pickup...,0.000000
3,Biang Kerok Shopee Tutup ; PHK di Mana-Mana Te...,-0.100000
4,"Lah itu shopee baru PHK karyawan nya , banyak ...",0.318182
5,Shopee bakar uangnya daripada ngundang artis K...,0.450000
6,"Lah itu shopee baru PHK karyawan nya , banyak ...",0.247273
7,"PHK Massal, Fenomena Tak Terhindarkan dalam Si...",0.107143
8,"Tergantung lingkungan juga sih, cowo gue ga la...",0.000000
9,Dan pada akhirnya pengguna shopee pada beralih...,0.137500


### Labelling

In [13]:
def labelling(polarity):
    if polarity > 0:
        return "Positif"
    elif polarity == 0:
        return "Netral"
    else:
        return "Negatif"
    
clean_data["Label"] = clean_data["Polarity"].apply(labelling)

In [14]:
clean_data["True Label"] = true_labels

In [15]:
clean_data

,text,Polarity,Label,True Label
0,PHK masal | Serba-serbi MMC Perusahaan besar y...,0.000000,Netral,Netral
1,"Gatau deh, Kaz. Semenjak berita yg shopee phk ...",0.500000,Positif,Negatif
2,kurir nya semua di PHK kah? Mau request pickup...,0.000000,Netral,Negatif
3,Biang Kerok Shopee Tutup ; PHK di Mana-Mana Te...,-0.100000,Negatif,Negatif
4,"Lah itu shopee baru PHK karyawan nya , banyak ...",0.318182,Positif,Negatif
5,Shopee bakar uangnya daripada ngundang artis K...,0.450000,Positif,Negatif
6,"Lah itu shopee baru PHK karyawan nya , banyak ...",0.247273,Positif,Negatif
7,"PHK Massal, Fenomena Tak Terhindarkan dalam Si...",0.107143,Positif,Negatif
8,"Tergantung lingkungan juga sih, cowo gue ga la...",0.000000,Netral,Netral
9,Dan pada akhirnya pengguna shopee pada beralih...,0.137500,Positif,Negatif


### Hasil Sentiment Analysis

In [16]:
label_counts = clean_data['Label'].value_counts()

print("Hasil Sentiment Analysis:\n")
for label, count in zip(label_counts.index, label_counts):
    print(f"{label}: {count} ({round(count/label_counts.sum(),2)*100}%)")

Hasil Sentiment Analysis:

Positif: 6 (50.0%)
Netral: 4 (33.0%)
Negatif: 2 (17.0%)


In [17]:
true_counts = clean_data['True Label'].value_counts()

print("Hasil Sebenarnya:\n")
for label, count in zip(true_counts.index, true_counts):
    print(f"{label}: {count} ({round(count/true_counts.sum(),2)*100}%)")

Hasil Sebenarnya:

Negatif: 9 (75.0%)
Netral: 3 (25.0%)


**Note:**<br>
Hasil labelling di atas **tidak sepenuhnya benar**. Berikut adalah alasannya:<br>
1. Beberapa **kata yang disingkat-singkat** sehingga TextBlob tidak mengerti sintaks dan semantik dari kalimat tertentu
2. Kalimat tertentu memiliki beberapa emoji dan kata positif (contoh: "baru") **dianggap memiliki label positif, meskipun seharusnya memberikan label negatif**
3. Hasil **translate** yang justru mungkin **memberikan interpretasi yang salah**

### Save dataframe to csv

In [18]:
clean_data.to_csv("twitter_sentiment_analysis.csv")